# Enviroment Setup

In [1]:
!git clone https://github.com/Kolibri25/WINDIFY_AI


Cloning into 'WINDIFY_AI'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 121 (delta 43), reused 77 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (121/121), 92.96 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
cd WINDIFY_AI

/content/WINDIFY_AI


Setup environment

In [6]:
!pip install -r requirements.txt

  Using cached netCDF4-1.7.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached xarray-2023.6.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached requests-2.28.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached matplotlib-3.6.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached dask-2023.6.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached torch-1.13.0-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Ignored the following versions that require a different python

In [8]:
!pip install netCDF4


  Using cached netCDF4-1.7.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.3 MB/s eta 0:00:00


In [9]:
import scripts.download_barra as dnb
import scripts.load_data as ld
from scripts.model_input import create_uv_tensor, create_uv_tensor_era5, split_tensor
import models.unet_model


In [10]:
import xarray as xr
import torch
import numpy as np
import matplotlib.pyplot as plt

Below purely to document BARRA data download and import procedure:

In [ ]:
'''

# download barrac2 data from months of Oct, Nov, Dec, Jan from 1979 to 2024
# lat/lon values and variables are hard-coded in the script
# need to check download path

dnb.main()

# download era5
# TODO: write script to download era5 data

'''

In [ ]:
'''
# combine files into single dataset per variable
# BARRA-C2
ld.load_and_save_barra_c2()
# ERA5
ld.load_and_save_era5()

Importing previously downloaded data using gdown from Google Drive folder

In [11]:
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/1xwqJ8Ku_US5Q7MRvExKC26TqCUroQ6QE?usp=sharing

Retrieving folder contents
Retrieving folder 1_rHNYO880eYBNB0w7ciNWKxCop2oq4Na ERA5
Processing file 1biSKBWzXjYk8e7UWYl-QJs-bP4z0FWqe combined_era5.nc
Retrieving folder 1f2DGtE-oKAuXlnp2re9byp4UbMitIFOf BARRA-C2
Processing file 1MPIFRs3SoilpeLTMn7P21NyiRAOyZNIB uas_gesamt.nc
Processing file 1hkWS9IlTFCIS0BGv0Yl0WbgkILRsaPa_ vas_gesamt.nc
Retrieving folder 1L-QsfJgqixDRWQHN6jxEkWdid0-YjZtX processed_data
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1biSKBWzXjYk8e7UWYl-QJs-bP4z0FWqe
To: /content/WINDIFY_AI/metfut_data/ERA5/combined_era5.nc
100% 70.3M/70.3M [00:00<00:00, 182MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1MPIFRs3SoilpeLTMn7P21NyiRAOyZNIB
From (redirected): https://drive.google.com/uc?id=1MPIFRs3SoilpeLTMn7P21NyiRAOyZNIB&confirm=t&uuid=926a494b-c93d-45bf-b6ef-2f4ea4cf2a75
To: /content/WINDIFY_AI/metfut_data/BARRA-C2/uas_gesamt.nc
100% 1.72G/1

In [12]:
!mv metfut_data/* /content/WINDIFY_AI/data/

In [13]:
# load target data (BARRA-C2)

uas_ds = xr.open_dataset('/content/WINDIFY_AI/data/BARRA-C2/uas_gesamt.nc')
vas_ds = xr.open_dataset('/content/WINDIFY_AI/data/BARRA-C2/vas_gesamt.nc')

# load input data (ERA5)

era5_ds = xr.open_dataset('/content/WINDIFY_AI/data/ERA5/combined_era5.nc')

# Data Preprocessing

Steps for Target ie. BARRA-C2:

1. Drop grid points to match model dimensions (divisible by 16).
2. Normalise the data.


Steps for Input ie. ERA5

1.   Drop grid points to match model dimensions (divisible by 16).
2.   Regrid EAR5 to a high-resolution grid of BARRA-C2
3.  Normalise the data.


Finally, we stack the input/target together as PyTorch tensors and split the data.

## Target: BARRA-C2

In [14]:
# crop to make dimensions divisible by 16

from scripts.crop_to_div16 import crop_to_multiple_of_16

uas_ds = crop_to_multiple_of_16(uas_ds, lat_name='lat', lon_name='lon')
vas_ds = crop_to_multiple_of_16(vas_ds, lat_name='lat', lon_name='lon')


In [16]:
# normalise the target data

from scripts.normalize_data import normalize_data

uas_ds, uas_mean, uas_std = normalize_data(uas_ds,"uas")
vas_ds, vas_mean, vas_std = normalize_data(vas_ds,"vas")

## Input: ERA5

In [18]:
# regrid to barra grid

era5_interp = era5_ds.interp(
    latitude=uas_ds.lat,
    longitude=uas_ds.lon,
    method='nearest'
)

In [20]:
# normalise input data

u10_ds, u10_mean, u10_std = normalize_data(era5_interp,"u10")
v10_ds, v10_mean, v10_std = normalize_data(era5_interp,"v10")

In [1]:
# convert to PyTorch tensors

print("Shape of uas_ds:", uas_ds.lon.shape)
print("Shape of vas_ds:", vas_ds.lon.shape)
print("uas_ds longitude values:", uas_ds.lon.values)
print("vas_ds longitude values:", vas_ds.lon.values)

# Interpolate vas_ds to match uas_ds longitude coordinates
vas_ds_interp = vas_ds.interp(lon=uas_ds.lon, method='nearest')

target = create_uv_tensor(uas_ds,vas_ds_interp)
input = create_uv_tensor_era5(u10_ds["u10"],v10_ds["v10"])

NameError: name 'uas_ds' is not defined

In [ ]:
# check shapes

print(target.shape)
print(input.shape)

In [ ]:
# save in case of runtime shortage

from google.colab import drive
drive.mount('/content/drive')

output_path = "/content/drive/MyDrive/metfut_data/processed_data/"

import os

# Ensure trailing slash and directory exists
os.makedirs(output_path, exist_ok=True)

# Use os.path.join to avoid hardcoding slashes
torch.save(target, os.path.join(output_path, 'target.pt'))
torch.save(input, os.path.join(output_path, 'input.pt'))



In [ ]:
# train/validation/test split

x_train, x_val, x_test = split_tensor(input)
y_train, y_val, y_test = split_tensor(target)